### **`check_envs_registration.ipynb`**


This notebook verifies the successful registration of all custom `LLECBuildingGym` environments and performs basic environment interaction tests.  
It includes:
- Import validation and environment listing  
- Basic reset and step checks for selected environments  
- Reward consistency validation  
- Gymnasium environment compliance check (`check_env`)

In [15]:
import sys
import numpy as np
import gymnasium as gym

print("Python executable:", sys.executable)
# Display all available Jupyter kernels
!jupyter kernelspec list | grep llec_env

# Attempt to the LLECBuildingGym module, which registers custom environments via `register()`
try:
    import llec_building_gym  # Triggers registration on llec_building_gym/__init__.py and registers environments

    print("llec_building_gym path:", llec_building_gym.__file__)
    print("Module 'llec_building_gym' successfully loaded.")
except ImportError:
    print("Module 'llec_building_gym' could not be loaded.")
    print("Please install it using: pip install -e .")
else:
    print("\nRegistered LLEC environments:")
    # List all environments containing 'LLEC' in their ID
    custom_envs = [
        env_id for env_id in sorted(gym.envs.registry.keys()) if "LLEC" in env_id
    ]
    if not custom_envs:
        print("No LLEC environments found.")
    else:
        for env_id in custom_envs:
            print(f"  {env_id}")
        print(f"\nTotal found: {len(custom_envs)} LLEC environments")

Python executable: /home/iai/ii6824/llec_env/bin/python
  python3           /hkfs/home/haicore/iai/ii6824/llec_env/share/jupyter/kernels/python3
llec_building_gym path: /hkfs/home/haicore/iai/ii6824/LLECBuildingGym/llec_building_gym/__init__.py
Module 'llec_building_gym' successfully loaded.

Registered LLEC environments:
  LLEC-HeatPumpHouse-1R1C-Combined-v0
  LLEC-HeatPumpHouse-1R1C-Temperature-v0

Total found: 2 LLEC environments


In [16]:
print(llec_building_gym.__file__)

# Choose one of the available environments below:
env = gym.make("LLEC-HeatPumpHouse-1R1C-Temperature-v0")
# env = gym.make("LLEC-HeatPumpHouse-1R1C-Combined-v0")

# Print observation and action space information
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)

# Reset the environment and display the initial observation
obs, info = env.reset()
print("Initial observation:", obs)

2025-11-20 21:08:42,020 - INFO - [EnvID:23409952903760] schedule=synthetic-temp-class class=Cold | Tmin=-1.3degC  Tmax=9.0degC  φ=0.137  seed=6184
2025-11-20 21:08:42,029 - INFO - [EnvID:23409952903760] schedule=synthetic-temp-class class=Cold | Tmin=-1.3degC  Tmax=9.0degC  φ=0.137  seed=8392


/hkfs/home/haicore/iai/ii6824/LLECBuildingGym/llec_building_gym/__init__.py
Observation Space: Box(-inf, inf, (1,), float32)
Action Space: Box(-1.0, 1.0, (1,), float32)
Initial observation: [1.8914707]


In [17]:
# Print all registered environments
print("=== Registered Gym Environments ===")
env_list = sorted(gym.envs.registry.keys())

# Check if the desired environment is registered
env_name = "LLEC-HeatPumpHouse-1R1C-Temperature-v0"
if env_name in env_list:
    print(f"\nEnvironment '{env_name}' is successfully registered.")
else:
    print(
        f"\nEnvironment '{env_name}' is not registered. Please check your setup or registration logic."
    )

=== Registered Gym Environments ===

Environment 'LLEC-HeatPumpHouse-1R1C-Temperature-v0' is successfully registered.


In [18]:
from llec_building_gym import BaseBuildingGym  # Adjust the path if necessary

# Instantiate the environment
env = BaseBuildingGym(energy_price_path="../data/price_data_2025.csv")
obs, _ = env.reset()
step_count = 0

# Simulate one full episode using random actions
while True:
    action = env.action_space.sample()  # Sample a random action
    obs, reward, terminated, truncated, info = env.step(action)
    step_count += 1
    if terminated or truncated:
        break

# Print episode summary
print(f"Episode finished after {step_count} steps.")
print(f"Final indoor temperature: {env.building.T_in:.2f}°C")
print(f"Final observation: {obs}")
print(f"Final observation length: {len(obs)}")
print(f"Final reward: {reward:.2f}")

2025-11-20 21:08:42,117 - INFO - [EnvID:23405731434208] schedule=synthetic-temp-class class=Cold | Tmin=-1.3degC  Tmax=9.0degC  φ=0.137  seed=6184
2025-11-20 21:08:42,145 - INFO - [EnvID:23405731434208] schedule=synthetic-temp-class class=Cold | Tmin=-1.3degC  Tmax=9.0degC  φ=0.137  seed=6184


Episode finished after 288 steps.
Final indoor temperature: 3.47°C
Final observation: [-18.820786]
Final observation length: 1
Final reward: 0.00


In [19]:
# List of all registered environment IDs to test
registered_envs = [
    "LLEC-HeatPumpHouse-1R1C-Combined-v0",
    "LLEC-HeatPumpHouse-1R1C-Temperature-v0",
]


def test_env(env_id, n_steps=3):
    print(f"\n=== Testing: {env_id} ===")
    try:
        env = gym.make(env_id, energy_price_path="../data/price_data_2025.csv")
        obs, info = env.reset(seed=42)
        print(f"Initial observation: {obs}")

        for step in range(n_steps):
            action = env.action_space.sample()
            obs, reward, terminated, truncated, info = env.step(action)

            print(f"\nStep {step + 1}")
            print(f"  Action: {action}")
            print(f"  Observation: {obs}")
            print(f"  Reward: {reward:.4f}")
            print(f"  Terminated: {terminated}, Truncated: {truncated}")
            print(f"  Info:")
            for k, v in info.items():
                if isinstance(v, float):
                    print(f"    {k}: {v:.4f}")
                else:
                    print(f"    {k}: {v}")

            # Optional: Reward consistency check
            if "reward_temperature" in info and "reward_energy" in info:
                reward_check = info["reward_temperature"] + info["reward_energy"]
                diff = abs(reward - reward_check)
                status = "ok" if diff <= 1e-3 else "(!)"
                print(f"    Reward consistency check: Δ = {diff:.6f} [{status}]")

            if terminated or truncated:
                print("  Episode ended early.")
                break

        env.close()

    except Exception as e:
        print(f"Error testing {env_id}: {e}")


if __name__ == "__main__":
    for env_id in registered_envs:
        test_env(env_id, n_steps=3)

2025-11-20 21:08:42,297 - INFO - [EnvID:23405731433200] schedule=synthetic-temp-class class=Cold | Tmin=-1.3degC  Tmax=9.0degC  φ=0.137  seed=6184
2025-11-20 21:08:42,325 - INFO - [EnvID:23405731433200] schedule=synthetic-temp-class class=Very cold | Tmin=-6.0degC  Tmax=3.9degC  φ=0.184  seed=42
2025-11-20 21:08:42,380 - INFO - [EnvID:23405731448576] schedule=synthetic-temp-class class=Cold | Tmin=-1.3degC  Tmax=9.0degC  φ=0.137  seed=6184
2025-11-20 21:08:42,408 - INFO - [EnvID:23405731448576] schedule=synthetic-temp-class class=Very cold | Tmin=-6.0degC  Tmax=3.9degC  φ=0.184  seed=42



=== Testing: LLEC-HeatPumpHouse-1R1C-Combined-v0 ===
Initial observation: [0.8960016]

Step 1
  Action: [0.703522]
  Observation: [6.542649]
  Reward: -0.1362
  Terminated: False, Truncated: False
  Info:
    temp_deviation: 6.0859
    action: [0.703522]
    T_out: -1.9394
    Q_HP_Max: 1500
    controlled_Q_HP: 1055.2830
    P_HP_el: 1055.2830
    E_HP_el_Wh: 87.9403
    cop_used: 1.0000
    reward: -0.1362
    reward_temperature: 0.0023
    reward_economic: -0.1385
    reward_temperature_norm: 0.0023
    reward_economic_norm: -0.1385
    history_temp_deviations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [20]:
from stable_baselines3.common.env_checker import check_env
from llec_building_gym import BaseBuildingGym  # Adjust the import path if necessary

# Create an instance of your custom environment
env = BaseBuildingGym(energy_price_path="../data/price_data_2025.csv")

# Run the environment check to validate compatibility with Stable Baselines3
check_env(env, warn=True, skip_render_check=True)

2025-11-20 21:08:42,473 - INFO - [EnvID:23405731441776] schedule=synthetic-temp-class class=Cold | Tmin=-1.3degC  Tmax=9.0degC  φ=0.137  seed=6184
2025-11-20 21:08:42,502 - INFO - [EnvID:23405731441776] schedule=synthetic-temp-class class=Hot | Tmin=19.5degC  Tmax=31.4degC  φ=-0.094  seed=0
2025-11-20 21:08:42,529 - INFO - [EnvID:23405731441776] schedule=synthetic-temp-class class=Cold | Tmin=-1.3degC  Tmax=9.0degC  φ=0.137  seed=6184
2025-11-20 21:08:42,556 - INFO - [EnvID:23405731441776] schedule=synthetic-temp-class class=Cold | Tmin=-1.3degC  Tmax=9.0degC  φ=0.137  seed=6184


In [21]:
print(
    "Environment check completed successfully in `check_envs_registration.ipynb`. The environment is compatible with Stable Baselines3."
)

Environment check completed successfully in `check_envs_registration.ipynb`. The environment is compatible with Stable Baselines3.
